## The Battle of Neighborhoods (Week 2)

My previous notebook stated that I would be exploring 5 different major cities to determine the best spot to open a pizza restaurant.
<br>
I will now be changing that to only exploring 2 major cities.
<br>
- **Philadelphia** and **Pittsburgh, PA**.

**Exploring the data:**

In [5]:
import pandas as pd
import numpy as np
import requests

**Finding center of cities:**

In [75]:
# ! pip install folium==0.7.0

In [60]:
from geopy.geocoders import Nominatim
import folium

In [9]:
cities_list = []

# cities = ['New York, NY', 'Philadelphia, PA', 'Boston, MA', 'Baltimore, MD', 'Pittsburgh, PA']
cities = ['Philadelphia, PA', 'Pittsburgh, PA']

for city in cities:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    cities_list.append([city.split(',')[0].upper(),[latitude, longitude]])
print(cities_list)

[['PHILADELPHIA', [39.9527237, -75.1635262]], ['PITTSBURGH', [40.4416941, -79.9900861]]]


**Setting up variables for query:**

In [63]:
CLIENT_ID = 'WUPUAH5PA54ZBTSDADZUGWT0ROUITGR5S1ZPV3EIV1DYIVP3'
CLIENT_SECRET = 'F0VENJWDIYB3JQYW3DOON402F0DK34O2MSR42145SCXS0UYM'
VERSION = '20201101'
LIMIT = 100
RADIUS = 8046 # meters
SEARCH_QUERY = 'Pizza'

In [11]:
for x in cities_list:
    LAT = x[1][0]
    LNG = x[1][1]
#     print(LAT,LNG)

In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LAT,LNG,SEARCH_QUERY,RADIUS,LIMIT)

In [13]:
results = requests.get(url).json()

In [14]:
venues = results['response']['venues']
dataframe = pd.json_normalize(venues)

<hr>

## Philadelphia's Restaurants:

**Exploring the Pizza restaurants in this city:**

In [64]:
phil = cities_list[0][1]
phil_lat = phil[0]
phil_lng = phil[1]
phil_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,phil_lat,phil_lng,SEARCH_QUERY,RADIUS,LIMIT)
results = requests.get(phil_url).json()
venues = results['response']['venues']
phil_df = pd.json_normalize(venues)

**Cleaning the phil_df dataframe:**

In [65]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in phil_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = phil_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [66]:
phil_df = dataframe_filtered

In [67]:
phil_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Jake's Pizza,Pizza Place,201 N Broad St,at Race St,39.956123,-75.162536,"[{'label': 'display', 'lat': 39.95612338396027...",387,19107,US,Philadelphia,PA,United States,"[201 N Broad St (at Race St), Philadelphia, PA...",NaN,4b3d274ff964a520138f25e3
1,Rex Pizza,Pizza Place,1526 Race St,NaN,39.956430,-75.165591,"[{'label': 'display', 'lat': 39.95642960191507...",448,19102,US,Philadelphia,PA,United States,"[1526 Race St, Philadelphia, PA 19102, United ...",NaN,4bae4b82f964a520c99e3be3
2,Joe's Pizza,Pizza Place,122 S 16th St,at Sansom St,39.950372,-75.167422,"[{'label': 'display', 'lat': 39.95037232183254...",423,19102,US,Philadelphia,PA,United States,"[122 S 16th St (at Sansom St), Philadelphia, P...",NaN,4ac641bef964a52066b320e3
3,Yunique Pizza,Pizza Place,1617 John F Kennedy Blvd,NaN,39.953912,-75.166949,"[{'label': 'display', 'lat': 39.95391172071889...",320,19103,US,Philadelphia,PA,United States,"[1617 John F Kennedy Blvd, Philadelphia, PA 19...",NaN,4c0fb8be98102d7f6fa9e506
4,Zio's Pizza,Pizza Place,111 S 13th St,13th & Sansom,39.950071,-75.161920,"[{'label': 'display', 'lat': 39.95007113634469...",325,19107,US,Philadelphia,PA,United States,"[111 S 13th St (13th & Sansom), Philadelphia, ...",NaN,4a9ff82af964a520c83d20e3


**Let's plot these restaurants on a map:**

In [76]:
phil_map = folium.Map(location=[phil_lat,phil_lng], zoom_start=14)

folium.CircleMarker(
    [phil_lat,phil_lng],
    radius=10,
    color='red',
    popup='Philadelphia, PA',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(phil_map)

for lat, lng, label in zip(phil_df.lat, phil_df.lng, phil_df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(phil_map)
    
phil_map

**KMeans Cluster to find the best spot for a delivery service:**

<hr>

## Pittsburgh's restaurants:**

**Exploring the Pizza restaurants in this city:**

In [69]:
pitt = cities_list[1][1]
pitt_lat = pitt[0]
pitt_lng = pitt[1]
pitt_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,pitt_lat,pitt_lng,SEARCH_QUERY,RADIUS,LIMIT)
results = requests.get(pitt_url).json()
venues = results['response']['venues']
pitt_df = pd.json_normalize(venues)

**Cleaning the data in pitt_df:**

In [70]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in pitt_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = pitt_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [71]:
pitt_df = dataframe_filtered

In [73]:
pitt_df.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Pizza Parma,Pizza Place,963 Liberty Ave,40.443770,-79.995820,"[{'label': 'display', 'lat': 40.44377, 'lng': ...",537,15222,US,Pittsburgh,PA,United States,"[963 Liberty Ave, Pittsburgh, PA 15222, United...",NaN,NaN,4b761d05f964a5208f3d2ee3
1,Pizza Milano,Pizza Place,1304 5th Ave,40.438582,-79.987962,"[{'label': 'display', 'lat': 40.4385821185373,...",390,15219,US,Pittsburgh,PA,United States,"[1304 5th Ave, Pittsburgh, PA 15219, United St...",NaN,NaN,4ade2a17f964a520807321e3
2,Genoa Pizza and Bar,Pizza Place,111 Market St,40.439001,-80.003671,"[{'label': 'display', 'lat': 40.43900068922535...",1189,15222,US,Pittsburgh,PA,United States,"[111 Market St, Pittsburgh, PA 15222, United S...",NaN,NaN,4b166f06f964a52005b923e3
3,Domino's Pizza,Pizza Place,300 6th Ave Uppr 100,40.441832,-79.999361,"[{'label': 'display', 'lat': 40.44183246264836...",785,15222,US,Pittsburgh,PA,United States,"[300 6th Ave Uppr 100, Pittsburgh, PA 15222, U...",NaN,NaN,4c7fdfe22f1c236ab0952143
4,Napoli's Pizza,Pizza Place,1525 5th Ave,40.438725,-79.984352,"[{'label': 'display', 'lat': 40.43872451782226...",587,15219,US,Pittsburgh,PA,United States,"[1525 5th Ave, Pittsburgh, PA 15219, United St...",NaN,NaN,4da518c24df00f31b175f0db


**Let's plot these restaurants on a map:**

In [74]:
pitt_map = folium.Map(location=[pitt_lat,pitt_lng], zoom_start=14)

folium.CircleMarker(
    [pitt_lat,pitt_lng],
    radius=10,
    color='red',
    popup='Pittsburgh, PA',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(pitt_map)

for lat, lng, label in zip(pitt_df.lat, pitt_df.lng, pitt_df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(pitt_map)
    
pitt_map

**Grabbing data from Foursquare API:**

In [21]:
columns = ['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address', 'location.lat', 'location.lng', 'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.cc', 'location.city', 'location.state', 'location.country', 'location.formattedAddress', 'delivery.id', 'delivery.url', 'delivery.provider.name', 'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes', 'delivery.provider.icon.name', 'venuePage.id', 'location.neighborhood', 'location.crossStreet']
df = pd.DataFrame(columns=columns)
for x in cities_list:
    LAT = x[1][0]
    LNG = x[1][1]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LAT,LNG,SEARCH_QUERY,RADIUS,LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df2 = pd.json_normalize(venues)
    df = df.append(df2, ignore_index=True)

In [24]:
df.head(3)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood,location.crossStreet
0,4f045eeb00399761c77301e3,Prince Street Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1604267677,False,27 Prince St,40.723093,-73.994527,"[{'label': 'display', 'lat': 40.72309326145674...",1506,...,"[27 Prince St (btwn Mott & Elizabeth St), New ...",264357,https://www.seamless.com/menu/prince-st-pizza-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,99329256,NaN,btwn Mott & Elizabeth St
1,4a4560a7f964a5201aa81fe3,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1604267677,False,2 Gold St,40.707634,-74.006996,"[{'label': 'display', 'lat': 40.7076335455641,...",573,...,"[2 Gold St (at Maiden Ln.), New York, NY 10038...",NaN,NaN,NaN,NaN,NaN,NaN,77559825,NaN,at Maiden Ln.
2,4af216e7f964a520d0e521e3,Dona Bella Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1604267677,False,154 Church Steet,40.714990,-74.007850,"[{'label': 'display', 'lat': 40.71499001654046...",295,...,"[154 Church Steet (Chambers St), New York, NY ...",69858,https://www.seamless.com/menu/little-italy-piz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,Chambers St


**Cleaning dataframe:**

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [26]:
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Prince Street Pizza,Pizza Place,27 Prince St,40.723093,-73.994527,"[{'label': 'display', 'lat': 40.72309326145674...",1506,10012,US,New York,NY,United States,"[27 Prince St (btwn Mott & Elizabeth St), New ...",NaN,btwn Mott & Elizabeth St,4f045eeb00399761c77301e3
1,Harry's Italian Pizza Bar,Italian Restaurant,2 Gold St,40.707634,-74.006996,"[{'label': 'display', 'lat': 40.7076335455641,...",573,10038,US,New York,NY,United States,"[2 Gold St (at Maiden Ln.), New York, NY 10038...",NaN,at Maiden Ln.,4a4560a7f964a5201aa81fe3
2,Dona Bella Pizza,Pizza Place,154 Church Steet,40.714990,-74.007850,"[{'label': 'display', 'lat': 40.71499001654046...",295,10007,US,New York,NY,United States,"[154 Church Steet (Chambers St), New York, NY ...",NaN,Chambers St,4af216e7f964a520d0e521e3
3,Little Italy Gourmet Pizza,Pizza Place,11 Park Pl,40.713044,-74.008334,"[{'label': 'display', 'lat': 40.713044, 'lng':...",198,10007,US,New York,NY,United States,"[11 Park Pl (Broadway and Church), New York, N...",NaN,Broadway and Church,4bf6ae6c94b2a59392deadee
4,Stage Door Pizza,Pizza Place,26 Vesey St,40.711838,-74.010189,"[{'label': 'display', 'lat': 40.71183761933663...",365,10007,US,New York,NY,United States,"[26 Vesey St, New York, NY 10007, United States]",NaN,NaN,52698af411d29a41c427892c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Pizza Parma,Pizza Place,823 Penn Ave,40.443886,-79.998573,"[{'label': 'display', 'lat': 40.44388580322265...",759,15222,US,Pittsburgh,PA,United States,"[823 Penn Ave, Pittsburgh, PA 15222, United St...",NaN,NaN,59f206191bc7045cb0fdef99
146,Pizza Vespucci,Pizza Place,1710 Forbes Ave,40.437377,-79.982265,"[{'label': 'display', 'lat': 40.43737700000000...",818,15219,US,Pittsburgh,PA,United States,"[1710 Forbes Ave, Pittsburgh, PA 15219, United...",NaN,NaN,51796804498e66dca6f16124
147,Pizza Hut,Pizza Place,430 Wood St,40.440151,-80.000542,"[{'label': 'display', 'lat': 40.44015121459961...",902,15222,US,Pittsburgh,PA,United States,"[430 Wood St, Pittsburgh, PA 15222, United Sta...",NaN,NaN,4f32500719836c91c7cb5cde
148,brother's pizza,None,Benner pike,40.440394,-79.995415,"[{'label': 'display', 'lat': 40.44039429610388...",474,NaN,US,Pittsburgh,PA,United States,"[Benner pike, Pittsburgh, PA, United States]",NaN,NaN,4b2c4201f964a520c3c424e3
